In [24]:
import numpy as np
import pandas as pd
import os
import re

In [25]:
#os.chdir('C:/Users/64672/Desktop/C题全部数据')

In [26]:
df=pd.read_excel('主题2.xlsx',encoding='gbk')

In [27]:
df1=df.loc[:,['留言编号','主题']]

In [28]:
df1['主题']=df1['主题'].apply(lambda x: re.sub('[^\u4E00-\u9FD5]+',',',x))

In [29]:
df1['主题']=df1['主题'].apply(lambda x: x[1:-1])

In [30]:
df1['主题']

0       业委会,业主大会,业主,意见,停车,情况,住房,坪,书记,工作,业委会,业主,业主大会,情况...
1       施工,项目,排水,道路,大道,土方,坪塘,渠道,管线,换填,施工,道路,排水,项目,集镇,换...
2       教师,幼儿园,民办,待遇,学前教育,依法,提高,监管,教职工,推进,待遇,民办,幼儿园,教师...
3       购房,市,补贴,首次,含,房屋交易,情况,万元,住房,商品,购房,市,含,首次,补贴,回复,...
4       马坡岭,来信,小学,站名,周边,收悉,市民,具体内容,变更,不宜,市民,小学,来信,马坡岭,...
                              ...                        
2811                收悉,人民政府,已转,区,调查,区委,区委,调查,区,已转,人民政府,收悉
2812                      局,交通运输,收悉,转市,调查,调查,转市,收悉,交通运输,局
2813    学生,社团,学校,社团活动,小学,课程,文盛,有利于,家长,调查,学生,社团活动,学校,社团...
2814    资金,支付,转移,燃油税,年,情况,市州,中央,养护,亿元,资金,支付,情况,燃油税,转移,...
2815    交通运输,项目,建设,前期工作,公路,单位,厅,规划,发展,工作进度,交通运输,项目,规划,...
Name: 主题, Length: 2816, dtype: object

In [31]:
df2=pd.read_excel('附件4.xlsx')

In [32]:
import jieba

In [33]:
df2['留言主题']=df2['留言主题'].apply(jieba.lcut)

In [34]:
df1['回复主题']=df1['主题'].apply(lambda x: x.split(','))

In [35]:
df1['留言主题']=df2['留言主题']

In [36]:
del df1['主题']

In [37]:
'''
with open('E:/MY_COMPETE/2020挑战杯/C题数据&代码/第三问/stopwords.txt','r',encoding='gbk') as f:
    stop=f.read()
stop=stop.split()
'''
stopWords = pd.read_csv('stopword.txt', encoding='GB18030', sep='hahaha', header=None)
#列表使用+拼接
stopWords = [' ', '\n', '\t', '\r\n', '\u3000', '＂', '–','现将','在','的','您好','你好','您','网友','留言','已收悉','：',
                '反映','的问题','平台','同志','现','领导'] + list(stopWords.iloc[:, 0])
df1['留言主题']=df1['留言主题'].apply(lambda x:[i for i in x if i not in stopWords])

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [38]:
#计算中文编辑距离 str1和str2 是经过分词的
def edit_similar(str1,str2):
    len_str1=len(str1)
    len_str2=len(str2)
    taglist=np.zeros((len_str1+1,len_str2+1))
    for a in range(len_str1):
        taglist[a][0]=a
    for a in range(len_str2):
        taglist[0][a] = a
    for i in range(1,len_str1+1):
        for j in range(1,len_str2+1):
            if(str1[i - 1] == str2[j - 1]):
                temp = 0
            else:
                temp = 1
            taglist[i][j] = min(taglist[i - 1][j - 1] + temp, taglist[i][j - 1] + 1, taglist[i - 1][j] + 1)
    return 1-taglist[len_str1][len_str2] / max(len_str1, len_str2)

In [39]:
#计算余弦相似度
def cos_dist(str1,str2):
    key_word = list(set(str1 + str2))
     # 给定形状和类型的用0填充的矩阵存储向量
    word_vector1 = np.zeros(len(key_word))
    word_vector2 = np.zeros(len(key_word))

    # 计算词频
    # 依次确定向量的每个位置的值
    for i in range(len(key_word)):
        # 遍历key_word中每个词在句子中的出现次数
        for j in range(len(str1)):
            if key_word[i] == str1[j]:
                word_vector1[i] += 1
        for k in range(len(str2)):
            if key_word[i] == str2[k]:
                word_vector2[i] += 1


    dist1=float(np.dot(word_vector1,word_vector2)/(np.linalg.norm(word_vector1)*np.linalg.norm(word_vector2)))
    return dist1

In [40]:
#余弦相似度赋予权值0.7，编辑距离赋予权重0.3
def compare(str1,str2):
    cos_result=cos_dist(str1,str2)
    edit_result=edit_similar(str1,str2)
    return cos_result*0.7 + edit_result*0.3

In [41]:
f_similar=[]
for i in range(len(df1)):
    a=compare(df1['留言主题'][i],df1['回复主题'][i])
    f_similar.append(a)

In [42]:
df1['相似度']=f_similar

In [43]:
df1.to_excel('相似度.xlsx',index=False)